# Cython for NumPy users

To follow the tutorial, see https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html

In [1]:
%load_ext cython
import Cython
print(Cython.__version__)

3.1.0a1


In [2]:
import numpy as np
array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
a = 4
b = 3
c = 9

### The first Cython program

##### Numpy version

In [3]:
def compute_np(array_1, array_2, a, b, c):
    return np.clip(array_1, 2, 10) * a + array_2 * b + c

In [4]:
timeit_result = %timeit -o compute_np(array_1, array_2, a, b, c)
np_time = timeit_result.average

6.94 ms ± 75.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
np_result = compute_np(array_1, array_2, a, b, c)

##### Pure Python version

In [6]:
def clip(a, min_value, max_value):
    return min(max(a, min_value), max_value)


def compute(array_1, array_2, a, b, c):
    """
    This function must implement the formula
    np.clip(array_1, 2, 10) * a + array_2 * b + c

    array_1 and array_2 are 2D.
    """
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]
    
    assert array_1.shape == array_2.shape

    result = np.zeros((x_max, y_max), dtype=array_1.dtype)

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result

In [7]:
assert np.all(compute(array_1, array_2, a, b, c) == np_result)

In [8]:
timeit_result = %timeit -o compute(array_1, array_2, a, b, c)
py_time = timeit_result.average

2.65 s ± 64.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### We make a function to be able to easily compare timings with the NumPy version and the pure Python version.

In [9]:
def compare_time(current, reference, name):
    ratio = reference/current
    if ratio > 1:
        word = "faster"
    else:
        ratio = 1 / ratio 
        word = "slower"
        
    print("We are", "{0:.1f}".format(ratio), "times", word, "than the", name, "version.")

def print_report(compute_function):
    assert np.all(compute_function(array_1, array_2, a, b, c) == np_result)
    timeit_result = %timeit -o compute_function(array_1, array_2, a, b, c)
    run_time = timeit_result.average
    compare_time(run_time, py_time, "pure Python")
    compare_time(run_time, np_time, "NumPy")

##### Pure Python version compiled with Cython:

In [10]:
%%cython -a
import numpy as np


def clip(a, min_value, max_value):
    return min(max(a, min_value), max_value)


def compute(array_1, array_2, a, b, c):
    """
    This function must implement the formula
    np.clip(array_1, 2, 10) * a + array_2 * b + c

    array_1 and array_2 are 2D.
    """
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]
    
    assert array_1.shape == array_2.shape

    result = np.zeros((x_max, y_max), dtype=array_1.dtype)

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result

In [11]:
print_report(compute)

1.92 s ± 50.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
We are 1.4 times faster than the pure Python version.
We are 276.8 times slower than the NumPy version.


### Adding types:

In [ ]:
%%cython -a
import numpy as np

# We now need to fix a datatype for our arrays. I've used the variable
# DTYPE for this, which is assigned to the usual NumPy runtime
# type info object.
DTYPE = np.intc

# cdef means here that this function is a plain C function (so faster).
# To get all the benefits, we type the arguments and the return value as int.
cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


def compute(array_1, array_2, int a, int b, int c):
    
    # The "cdef" keyword is also used within functions to type variables. It
    # can only be used at the top indentation level (there are non-trivial
    # problems with allowing them in other places, though we'd love to see
    # good and thought out proposals for it).
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    
    assert array_1.shape == array_2.shape
    assert array_1.dtype == DTYPE
    assert array_2.dtype == DTYPE

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    
    # It is very important to type ALL your variables. You do not get any
    # warnings if not, only much slower code (they are implicitly typed as
    # Python objects).
    # For the "tmp" variable, we want to use the same data type as is
    # stored in the array, so we use int because it correspond to np.intc.
    # NB! An important side-effect of this is that if "tmp" overflows its
    # datatype size, it will simply wrap around like in C, rather than raise
    # an error like in Python.

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result

In [13]:
print_report(compute)

1.7 s ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
We are 1.6 times faster than the pure Python version.
We are 245.2 times slower than the NumPy version.


### Efficient indexing with memoryviews:

In [14]:
%%cython -a
import numpy as np

DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


def compute(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):
     
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [15]:
print_report(compute)

5.9 ms ± 312 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
We are 449.8 times faster than the pure Python version.
We are 1.2 times faster than the NumPy version.


### Tuning indexing further:

In [16]:
%%cython -a
import numpy as np
cimport cython

DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)

@cython.boundscheck(False)
@cython.wraparound(False)
def compute(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):
     
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [17]:
print_report(compute)

4.11 ms ± 45.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
We are 645.3 times faster than the pure Python version.
We are 1.7 times faster than the NumPy version.


### Declaring the NumPy arrays as contiguous.

In [18]:
%%cython
import numpy as np
cimport cython

DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute(int[:, ::1] array_1, int[:, ::1] array_2, int a, int b, int c):
     
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, ::1] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [19]:
print_report(compute)

1.04 ms ± 15.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
We are 2539.8 times faster than the pure Python version.
We are 6.6 times faster than the NumPy version.


### Making the function cleaner

In [20]:
%%cython -a
# cython: infer_types=True
import numpy as np
cimport cython

DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute(int[:, ::1] array_1, int[:, ::1] array_2, int a, int b, int c):
     
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]
    
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, ::1] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [21]:
print_report(compute)

1.04 ms ± 55.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
We are 2555.5 times faster than the pure Python version.
We are 6.7 times faster than the NumPy version.


### More generic code:

In [22]:
%%cython
# cython: infer_types=True
import numpy as np
cimport cython

ctypedef fused my_type:
    int
    double
    long long


cdef my_type clip(my_type a, my_type min_value, my_type max_value):
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute(my_type[:, ::1] array_1, my_type[:, ::1] array_2, my_type a, my_type b, my_type c):
     
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]
    
    assert tuple(array_1.shape) == tuple(array_2.shape)
    
    if my_type is int:
        dtype = np.intc
    elif my_type is double:
        dtype = np.double
    elif my_type is cython.longlong:
        dtype = np.double
        
    result = np.zeros((x_max, y_max), dtype=dtype)
    cdef my_type[:, ::1] result_view = result

    cdef my_type tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [23]:
arr_1_float = array_1.astype(np.float64)
arr_2_float = array_2.astype(np.float64)

float_cython_result = compute(arr_1_float, arr_2_float, a, b, c)
float_numpy_result = compute_np(arr_1_float, arr_2_float, a, b, c)

assert np.all(float_cython_result == float_numpy_result)

In [24]:
print_report(compute)

7.5 ms ± 217 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
We are 353.9 times faster than the pure Python version.
We are 1.1 times slower than the NumPy version.


### Using multiple threads

In [25]:
%%cython --force
# distutils: extra_compile_args=-Xpreprocessor -fopenmp
# distutils: extra_link_args=-L/opt/homebrew/Cellar/libomp/19.1.6/lib -lomp
# distutils: include_dirs=/opt/homebrew/Cellar/libomp/19.1.6/include



import numpy as np
cimport cython
from cython.parallel import prange

ctypedef fused my_type:
    int
    double
    long long


# We declare our plain c function nogil
cdef my_type clip(my_type a, my_type min_value, my_type max_value) noexcept nogil:
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute(my_type[:, ::1] array_1, my_type[:, ::1] array_2, my_type a, my_type b, my_type c):
     
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    
    assert tuple(array_1.shape) == tuple(array_2.shape)
    
    if my_type is int:
        dtype = np.intc
    elif my_type is double:
        dtype = np.double
    elif my_type is cython.longlong:
        dtype = np.longlong
        
    result = np.zeros((x_max, y_max), dtype=dtype)
    cdef my_type[:, ::1] result_view = result

    cdef my_type tmp
    cdef Py_ssize_t x, y

    # We use prange here.
    for x in prange(x_max, nogil=True):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

Content of stderr:
/Users/sebastianperalta/.cache/ipython/cython/_cython_magic_0e639bf8652694f670d3a551c0b507af6db095472d4293c263d9f7c7a0b10caa.c:27688:54: warning: code will never be executed [-Wunreachable-code]
 27688 |             if (__Pyx_PyList_SET_ITEM(list, i, string) < 0) goto __pyx_err;
       |                                                             ^~~~~~~~~~~~~~
/Users/sebastianperalta/.cache/ipython/cython/_cython_magic_0e639bf8652694f670d3a551c0b507af6db095472d4293c263d9f7c7a0b10caa.c:27688:51: note: silence by adding parentheses to mark code as explicitly dead
 27688 |             if (__Pyx_PyList_SET_ITEM(list, i, string) < 0) goto __pyx_err;
       |                                                          ^
       |                                                          /* DISABLES CODE */ ( )
/Users/sebastianperalta/.cache/ipython/cython/_cython_magic_0e639bf8652694f670d3a551c0b507af6db095472d4293c263d9f7c7a0b10caa.c:27755:60: warning: code will never be exec

In [26]:
print_report(compute)

752 μs ± 6.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
We are 3528.6 times faster than the pure Python version.
We are 9.2 times faster than the NumPy version.
